In [1]:
# objective is to create a game on a board where you can move and shoot
# We will need a player that is an element that can change places in an array
# The player will need to be able to control when they shoot
# let us begin with a simple version of an element moving in a 1d array

In [2]:
import numpy as np
import time
from IPython.display import clear_output

In [3]:
shot_dict = {"w":[0,1], "s":[0,-1], "d":[1,0], "a":[-1,0]}

In [4]:
def check_wall(var, end, velocity = 0, bounce = False):
    end = end - 1 # g = [0] * 8 has g[7] as last value
    var_greater = var > end
    var_lesser = var < 0
    if bounce is True:
        if var_greater:
            var = end - (var-end)
            velocity = -1 * velocity
        elif var_lesser:
            var = -1 * var
            velocity = -1 * velocity
        return var, velocity
    else:
        if var_greater:
             var = end
        elif var_lesser:
            var = 0
        return var

In [5]:
def rd(start = -2,end = 2):
    return np.random.randint(start,end+1)

In [121]:
def shooting(hum, bees, board, counter, max_velocity = 3):
    if hum.health == True:
        shoot = input(hum.name + " shoot: ")
        if len(shoot) != 0:
            shot = [sum(x) for x in zip(*[shot_dict[i] for i in shoot])]
            if not (sum(shot) > max_velocity):
                b = Bullet(hum,*shot,"*",board, ID = hum.shape + str(counter))
                bees.append(b)

In [122]:
def collision(hums, bees, board):
    hums_del, bees_del = [], []
    bees_positions = [b.position for b in bees]
    hums_positions = [hum.position for hum in hums]
    positions = hums_positions + bees_positions
    for position in positions:
        duplicates = [index for index,element in enumerate(positions) if element == position]
        if len(duplicates) > 1:
            print(duplicates)
            for duplicate in duplicates:
                if duplicate < len(hums_positions):
                    hums_del.append(duplicate)
                else:
                    bees_del.append(duplicate)
                    
    bees = [b for index, b in enumerate(bees) if index + len(hums) not in bees_del]
    hums = [hum for index, hum in enumerate(hums) if index not in hums_del]
    return hums, bees

In [123]:
def moving(hum, board):
    if hum.health == True:
        direction = input(hum.name + " move: ")
        if direction in "wasd": hum.move(direction)

In [124]:
class Board():
    def __init__(self,L,H,shape):
        self.L = L
        self.H = H
        self.shape = shape
        self.board = [[shape]*L for i in range(H)]
        
    def update(self, *actors):
        self.clear()
        for actor in actors:
            self.board[(self.H - 1) - actor.y][actor.x] = actor.shape
        
    def show(self):
        for i in range(self.H): print(*self.board[i])
            
    def clear(self):
        self.board = [[0]*self.L for i in range(self.H)]
    

In [125]:
class Human():
    def __init__(self,shape,x,y,board,name, health = True):
        self.shape = shape
        self.x = x
        self.y = y
        self.position = (x,y)
        self.board = board
        self.name = name
        self.health = health

    def move(self,key):
        if key == "w": self.y += 1
        elif key == "s": self.y -= 1
        elif key == "d": self.x += 1
        elif key == "a": self.x -= 1
        self.position = (self.x,self.y)
        self.x = check_wall(self.x, self.board.L)
        self.y = check_wall(self.y, self.board.H)

In [126]:
class Bullet():
    def __init__(self,actor,v_x,v_y,shape,board, ID):
        self.v_x = v_x
        self.v_y = v_y
        self.shape = shape
        self.x = actor.x
        self.y = actor.y
        self.position = (actor.x, actor.y)
        self.board = board
        self.ID = ID
    
    def fly(self):
        self.x += self.v_x
        self.y += self.v_y
        self.x, self.v_x = check_wall(self.x, self.board.L, self.v_x, bounce = True)
        self.y, self.v_y = check_wall(self.y, self.board.H, self.v_y, bounce = True)
        self.position = (self.x, self.y)

In [127]:
def turn(hum):
    moving(hum, board)
    shooting(hum, bees, board, max_velocity= 3)

In [128]:
board = Board(5, 5, shape = 0)
hum1 = Human("A",0,0,board,"player A")
hum2 = Human("B",board.L-1,board.H-1,board, "player B")
hum3 = Human("C",0,board.H-1,board, "player C")
hum4 = Human("D",board.L-1,0,board, "player D")
hums = [hum1, hum2, hum3, hum4]
counter = 0
bees = []
board.update(*hums)
board.show()

C 0 0 0 B
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
A 0 0 0 D


In [129]:
for i in range(15):
    counter += 1
    for hum in hums: turn(hum)
    for b in bees: b.fly()
    hums, bees = collision(hums, bees, board)
    board.update(*hums, *bees)
    clear_output()
    board.show()
    print("updated --------")
    time.sleep(0.5)

0 * 0 0 0
* 0 * 0 0
0 0 * 0 0
0 0 0 0 0
0 0 0 0 0
updated --------
